## GeoPandas Demo: Get Counties
This example demonstrates how to grab data from an ArcGIS MapService and pull it into a GeoPandas data frame.

In [ ]:
import requests
import pandas as pd
import geopandas as gpd

from shapely.geometry import LinearRing
from shapely.geometry import Polygon

%matplotlib inline

In [ ]:
#Specify the state FIPS to extact
stateFIPS = '37' #This is NC

In [ ]:
#Build the request and parameters to fetch county features
#  from the NOAA ArcGIS map server end point
url = 'https://nowcoast.noaa.gov/arcgis/rest/services/nowcoast/mapoverlays_political/MapServer/find'
params = {'searchText':stateFIPS,
          'contains':'true',
          'searchFields':'STATEFP',
          'sr':'',
          'layers':'2',
          'layerDefs':'',
          'returnGeometry':'true',
          'maxAllowableOffset':'',
          'geometryPrecision':'',
          'dynamicLayers':'',
          'returnZ':'false',
          'returnM':'false',
          'gdbVersion':'',
          'returnUnformattedValues':'false',
          'returnFieldName':'false',
          'datumTransformations':'',
          'layerParameterValues':'',
          'mapRangeValues':'',
          'layerRangeValues':'',
          'f':'json'}

In [ ]:
#Fetch the data
response = requests.get(url,params)

In [ ]:
#Convert to a JSON object (i.e. a dictionary)
results = response.json()['results']

In [ ]:
#Create a dataFrame from the results, 
#  keeping just the attributes and geometry objects
df = pd.DataFrame(results,columns=('attributes','geometry'))

In [ ]:
#Explode the dictionary values into fields
dfCounties = df['attributes'].apply(pd.Series)
dfGeom = df['geometry'].apply(pd.Series)

In [ ]:
#Combine the two
dfAll = pd.concat((dfCounties,dfGeom),axis='rows')

In [ ]:
#Define a function to convert a JSON 'ring' to a shapely polygon 
def polyFromRing(ring):
    r = LinearRing(ring)
    s = Polygon(r)
    return r

In [ ]:
#Iterate through all records and create a 'geometry object'
dfAll['geometry']=dfAll.apply(lambda x: Polygon(x.rings[0]),axis=1)

In [ ]:
#Convert the pandas dataFrame to a geopandas data frame
gdf=gpd.GeoDataFrame(dfAll)

In [ ]:
#Write to a csv file
gdf.to_csv("../Data/counties_{}.csv".format(stateFIPS))